In [ ]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.applications import MobileNetV2
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

import matplotlib.pyplot as plt
import cv2

In [ ]:
image_size = (64, 64, 3)

datagen = ImageDataGenerator(
    rescale = 1./255, zoom_range = 0.2, horizontal_flip = True, validation_split=0.1,
)

In [ ]:
training_set = datagen.flow_from_directory(
    'dataset/train-balanced/', target_size = image_size[:2],  batch_size = 32, class_mode = 'categorical', subset='training', 
    color_mode='rgb'
)

In [ ]:
validation_set = datagen.flow_from_directory(
    'dataset/train-balanced/',  target_size = image_size[:2], batch_size = 32, class_mode = 'categorical', subset='validation', 
    color_mode='rgb'
)

In [ ]:
training_set.class_indices

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=7)
filepath = "model.h5"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3)

In [ ]:
base_model = MobileNetV2(input_shape=image_size, include_top=False, weights='imagenet')
base_model.trainable = False

base_model.summary()

global_average_layer = GlobalAveragePooling2D()

prediction_layer = Dense(3, activation = 'softmax')

In [ ]:
neuralnetwork = Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

neuralnetwork.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

neuralnetwork.summary()

In [ ]:
history = neuralnetwork.fit_generator(
    training_set, steps_per_epoch = 108,
    validation_data = validation_set, validation_steps = 12,
    callbacks=[es, ckpt, rlp], epochs = 1000, 
)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 12))

fig.suptitle('Model Performance', fontsize=24) 

ax[0].plot(history.history['loss'], label='t-loss')
ax[0].plot(history.history['val_loss'], label='v-loss')
ax[0].set_title('Loss', fontsize=18)
ax[0].set_ylabel('Loss')

ax[1].plot(history.history['acc'], label='t-acc')
ax[1].plot(history.history['val_acc'], label='v-acc')
ax[1].set_title('Score', fontsize=18)
ax[1].set_ylabel('Score')


for i in range(2):
    ax[i].grid()
    ax[i].legend()
    ax[i].set_xlabel('Epochs')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import seaborn as sns

In [ ]:
preds = neuralnetwork.predict_generator(validation_set)
y_true = validation_set.classes
y_pred = preds.argmax(axis=1)


sns.heatmap(
    normalize(confusion_matrix(y_true, y_pred), axis=1, norm='l1') * 100, cmap='Blues', annot=True, 
    xticklabels=['with_mask', 'without_mask'],
    yticklabels=['with_mask', 'without_mask'],
    square=True
)